# Assignment: Segmenting and Clustering Neighbordhoods in Toronto

## Creating the Dataframe

### Import the Table

In [79]:
# import the library we use to open URLs
import urllib.request
import numpy as np
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"


In [80]:
# open the url using the library 

page = urllib.request.urlopen(url)


In [81]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(page, "lxml")

In [82]:
#print(soup.prettify())

In [83]:
soup.title.string


'List of postal codes of Canada: M - Wikipedia'

In [84]:
all_tables=soup.find_all("table")
#all_tables

In [85]:
right_table=soup.find('table',class_='wikitable sortable')

#right_table

In [86]:
a = []
b = []
c = []

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        a.append(cells[0].find(text=True))
        b.append(cells[1].find(text=True))
        c.append(cells[2].find(text=True))


### Clean the Table

Create the dataframe for the table

In [87]:
import pandas as pd

df=pd.DataFrame(a,columns=['Postal Code'])
df['Borough'] = b
df['Neighborhood'] = c

df


,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n
...,...,...,...
175,M5Z\n,Not assigned\n,\n
176,M6Z\n,Not assigned\n,\n
177,M7Z\n,Not assigned\n,\n
178,M8Z\n,Etobicoke\n,Mimico NW / The Queensway West / South of Bloo...


Remove all the \n

In [88]:

for col in df.columns:
    df[col] = df[col].map(lambda x: x.rstrip('\n'))

df.head()



,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Remove the "Not Assigned" Borough's

In [89]:

df['Borough'].replace('Not assigned',np.nan,inplace=True)
df.dropna(subset=['Borough'],inplace=True)
df

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
160,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


Replace all of the '/' with ',', I just did it to look like the table given. Since all the Postal Codes are already combined

In [90]:
df['Neighborhood']=df.Neighborhood.str.replace(' /', ',')
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


The final shape of the borough is found with:

In [91]:
df.shape


(103, 3)

In [92]:
df.sort_values('Postal Code', inplace = True)
df.reset_index(inplace = True)
df

,index,Postal Code,Borough,Neighborhood
0,9,M1B,Scarborough,"Malvern, Rouge"
1,18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,36,M1G,Scarborough,Woburn
4,45,M1H,Scarborough,Cedarbrae
...,...,...,...,...
98,98,M9N,York,Weston
99,107,M9P,Etobicoke,Westmount
100,116,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,143,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [93]:
df_latlong = pd.read_csv('http://cocl.us/Geospatial_data')
df_latlong

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [94]:
df.drop(['index'],axis = 1, inplace=True)

In [95]:
df['Latitude'] = df_latlong['Latitude']
df['Longitude'] = df_latlong['Longitude']

df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
